# A-B Testing for ShoeFly.com
Our favorite online shoe store, ShoeFly.com is performing an A/B Test. They have two different versions of an ad, which they have placed in emails, as well as in banner ads on Facebook, Twitter, and Google. They want to know how the two ads are performing on each of the different platforms on each day of the week. Help them analyze the data using aggregate measures.

## Analyzing Ad Sources

### 1.
Examine the first few rows of `ad_clicks`.

In [17]:
import pandas as pd
from IPython.display import display # Jupyter Notebooks render HTML tables

ad_clicks = pd.read_csv('ad_clicks.csv')
display(ad_clicks.head())

,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B


### 2.
Your manager wants to know which ad platform is getting you the most views.

How many views (i.e., rows of the table) came from each `utm_source`?

In [18]:
display(ad_clicks.groupby('utm_source').user_id.count())

utm_source
email       255
facebook    504
google      680
twitter     215
Name: user_id, dtype: int64

### 3.
If the column `ad_click_timestamp` is not null, then someone actually clicked on the ad that was displayed.

Create a new column called `is_click`, which is "True" if `ad_click_timestamp` is not null and "False" otherwise.

In [19]:
ad_clicks['is_click'] = ~ad_clicks.ad_click_timestamp.isnull()

### 4.
We want to know the percent of people who clicked on ads from each `utm_source`.

Start by grouping by `utm_source` and `is_click` and counting the number of `user_id`s in each of those groups. Save your answer to the variable `clicks_by_source`.

In [20]:
clicks_by_source = ad_clicks.groupby(['utm_source','is_click']).user_id.count().reset_index()
display(clicks_by_source)

,utm_source,is_click,user_id
0,email,False,175
1,email,True,80
2,facebook,False,324
3,facebook,True,180
4,google,False,441
5,google,True,239
6,twitter,False,149
7,twitter,True,66


### 5.
Now let’s pivot the data so that the columns are `is_click` (either "True" or "False"), the index is `utm_source`, and the values are `user_id`.

Save your results to the variable `clicks_pivot`.

In [21]:
clicks_pivot = clicks_by_source.pivot(columns='is_click',index='utm_source',values='user_id').reset_index()

display(clicks_pivot)

is_click,utm_source,False,True
0,email,175,80
1,facebook,324,180
2,google,441,239
3,twitter,149,66


### 6.
Create a new column in `clicks_pivot` called `percent_clicked` which is equal to the percent of users who clicked on the ad from each `utm_source`.

Was there a difference in click rates for each source?

In [22]:
clicks_pivot['percent_clicked'] = clicks_pivot[True] / (clicks_pivot[True] + clicks_pivot[False])
display(clicks_pivot)

is_click,utm_source,False,True,percent_clicked
0,email,175,80,0.313725
1,facebook,324,180,0.357143
2,google,441,239,0.351471
3,twitter,149,66,0.306977


## Analyzing an A/B Test

### 7.
The column `experimental_group` tells us whether the user was shown Ad A or Ad B.

Were approximately the same number of people shown both ads?

In [23]:
display(ad_clicks.groupby('experimental_group').user_id.count().reset_index())

,experimental_group,user_id
0,A,827
1,B,827


### 8.

Using the column `is_click` that we defined earlier, check to see if a greater percentage of users clicked on Ad A or Ad B.

In [24]:
display(ad_clicks.groupby(['experimental_group','is_click']).user_id.count().reset_index().pivot(index='experimental_group', columns='is_click', values='user_id').reset_index())


is_click,experimental_group,False,True
0,A,517,310
1,B,572,255


### 9.
The Product Manager for the A/B test thinks that the clicks might have changed by day of the week.

Start by creating two DataFrames: `a_clicks` and `b_clicks`, which contain only the results for A group and B group, respectively.

In [25]:
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']
display(a_clicks.head(5))
display(b_clicks.head(5))

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
5,013b0072-7b72-40e7-b698-98b4d0c9967f,facebook,1 - Monday,NaN,A,False
6,0153d85b-7660-4c39-92eb-1e1acd023280,google,4 - Thursday,NaN,A,False
7,01555297-d6e6-49ae-aeba-1b196fdbb09f,google,3 - Wednesday,NaN,A,False


,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False
9,01a210c3-fde0-4e6f-8efd-4f0e38730ae6,email,2 - Tuesday,15:21,B,True
10,01adb2e7-f711-4ae4-a7c6-29f48457eea1,google,3 - Wednesday,NaN,B,False


### 10.
For each group (`a_clicks` and `b_clicks`), calculate the percent of users who clicked on the ad by day.

In [26]:
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']

a_clicks_pivot = a_clicks.groupby(['is_click', 'day']).user_id.count().reset_index().pivot(
    index = 'day',
    columns = 'is_click',
    values = 'user_id',
).reset_index()

a_clicks_pivot['percent_clicked'] = a_clicks_pivot[True] / (a_clicks_pivot[True] + a_clicks_pivot[False])

display(a_clicks_pivot)

is_click,day,False,True,percent_clicked
0,1 - Monday,70,43,0.380531
1,2 - Tuesday,76,43,0.361345
2,3 - Wednesday,86,38,0.306452
3,4 - Thursday,69,47,0.405172
4,5 - Friday,77,51,0.398438
5,6 - Saturday,73,45,0.381356
6,7 - Sunday,66,43,0.394495


In [27]:
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']

b_clicks_pivot = b_clicks.groupby(['is_click', 'day']).user_id.count().reset_index().pivot(
    index = 'day',
    columns = 'is_click',
    values = 'user_id',
).reset_index()

b_clicks_pivot['percent_clicked'] = b_clicks_pivot[True] / (b_clicks_pivot[True] + b_clicks_pivot[False])

display(b_clicks_pivot)

is_click,day,False,True,percent_clicked
0,1 - Monday,81,32,0.283186
1,2 - Tuesday,74,45,0.378151
2,3 - Wednesday,89,35,0.282258
3,4 - Thursday,87,29,0.250000
4,5 - Friday,90,38,0.296875
5,6 - Saturday,76,42,0.355932
6,7 - Sunday,75,34,0.311927


### 11.
Compare the results for A and B. What happened over the course of the week?

Do you recommend that your company use Ad A or Ad B?

Ad A has a better click rate than B on every day of the week except for Tuesdays. A is looking like the more effective ad at driving clicks.